In [2]:
import pandas as pd
import numpy as np
import ast 

In [3]:
df_credits = pd.read_csv("../data_local/credits.csv")
df_movies = pd.read_csv("../data_local/movies_dataset.csv")

C:\Users\fate8\AppData\Local\Temp\ipykernel_5200\3265436363.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv("../data_local/movies_dataset.csv")


### Columnas que contienen dict o list
- belong_to_collection
- genres
- production_companies
- production_countries
- spoken_languages


### Datos necesarios para las funciones (movies)
-  #de peliculas estrenadas x mes
-  #de peliculas estrenadas x dia 
-  title, release_date, popularity
-  title, ,release_date, vote_count, vote_average
### Datos necesarios para las funciones (credits)
- name (cast, crew), department 
- budget(costo), revenue(ganancia) 

### Datos necesarios para unir los datasets
- id

In [4]:
df_movies['production_companies']

0           [{'name': 'Pixar Animation Studios', 'id': 3}]
1        [{'name': 'TriStar Pictures', 'id': 559}, {'na...
2        [{'name': 'Warner Bros.', 'id': 6194}, {'name'...
3        [{'name': 'Twentieth Century Fox Film Corporat...
4        [{'name': 'Sandollar Productions', 'id': 5842}...
                               ...                        
45461                                                   []
45462               [{'name': 'Sine Olivia', 'id': 19653}]
45463    [{'name': 'American World Pictures', 'id': 6165}]
45464                 [{'name': 'Yermoliev', 'id': 88753}]
45465                                                   []
Name: production_companies, Length: 45466, dtype: object

In [5]:
#Convertimos las filas de strings en estructuras de datos
#Extraemos el  ID y el NAME si es un diccionario valido, de lo contrario añadimos un 0

#belong_to_collection
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].apply(lambda x: ast.literal_eval(x)     if isinstance(x, str) else x)
df_movies['id_collection'] = df_movies['belongs_to_collection'].apply(lambda x: x.get('id') if isinstance(x, dict) else 0    )
df_movies['name_collection'] = df_movies['belongs_to_collection'].apply(lambda x: x.get('name') if isinstance(x, dict) else 0    )

#production_companies
df_movies['production_companies'] = df_movies['production_companies'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df_movies['id_companies'] = df_movies['production_companies'].apply(lambda x: x[0]['id'] if isinstance(x, list) and len(x) > 0 and isinstance(x[0], dict) else 0)
df_movies['name_companies'] = df_movies['production_companies'].apply(lambda x: x[0]['name'] if isinstance(x, list) and len(x) > 0 and isinstance(x[0], dict) else 0)

#production countries

#spoken_languages

## ETL movies

In [6]:
df_movies_limpio = df_movies.copy()

### **drop columns**

In [7]:
df_movies_limpio = df_movies_limpio.drop(columns=['adult','belongs_to_collection','genres','homepage','imdb_id','original_language','original_title','overview','poster_path','production_companies','production_countries','runtime','spoken_languages','runtime','status','tagline','video'])

### **order**

In [8]:
new_order = ['id','title','release_date','popularity','vote_average','vote_count','id_collection','name_collection','id_companies','name_companies','revenue','budget']
df_movies_limpio = df_movies_limpio[new_order]

### **rename**

In [9]:
df_movies_limpio.rename(columns={'id_collection': 'collection_id','name_collection':'collection_name','id_companies':'company_id','name_companies':'company_name'}, inplace=True)

###  *copia de seguridad*

In [10]:
df_movies_limpio_copia = df_movies_limpio.copy() 

### **nulls drop**

In [11]:
df_movies_limpio = df_movies_limpio.dropna(subset=['release_date'])

### **nullos por 0 (revenue-badget)**

In [12]:
df_movies_limpio['revenue'] = df_movies_limpio['revenue'].fillna(0)
df_movies_limpio['budget'] = df_movies_limpio['budget'].fillna(0)

### **release date format**

In [13]:
meses_español = [
    'enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio',
    'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre'
]

dias_espanol = [
    'lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo'
]

In [14]:
mask = pd.to_datetime(df_movies_limpio['release_date'], format= '%Y-%m-%d', errors= 'coerce').notna()
df_movies_limpio = df_movies_limpio[mask]
df_movies_limpio['release_date'] = pd.to_datetime(df_movies_limpio['release_date'], format='%Y-%m-%d')
df_movies_limpio['release_year'] = df_movies_limpio['release_date'].dt.year
df_movies_limpio['release_month'] = df_movies_limpio['release_date'].dt.month.apply(lambda x: meses_español[x-1])
df_movies_limpio['release_day'] = df_movies_limpio['release_date'].dt.dayofweek.apply(lambda x: dias_espanol[x])


### **id-title-popularity-vote_count format**

In [17]:
df_movies_limpio['id'] = df_movies_limpio['id'].astype(int)
df_movies_limpio['title'] = df_movies_limpio['title'].astype(str)
df_movies_limpio['popularity'] = df_movies_limpio['popularity'].astype(float)
df_movies_limpio['vote_count'] = df_movies_limpio['vote_count'].astype(int)
df_movies_limpio['revenue'] = df_movies_limpio['revenue'].astype(int)
df_movies_limpio['budget'] = df_movies_limpio['budget'].astype(int)
df_movies_limpio['return'] = df_movies_limpio['return'].apply(lambda x: format(x, '.3f'))


In [16]:
df_movies_limpio['return'] = (df_movies_limpio['revenue']/df_movies_limpio['budget'].replace(0,1)).round(3)

In [18]:
df_movies_limpio.dtypes

id                          int64
title                      object
release_date       datetime64[ns]
popularity                float64
vote_average              float64
vote_count                  int64
collection_id               int64
collection_name            object
company_id                  int64
company_name               object
revenue                     int64
budget                      int64
release_year                int32
release_month              object
release_day                object
return                     object
dtype: object

In [19]:
df_credits_crew = df_credits.copy()
df_credits_cast = df_credits.copy()


### **Extraccion crew_name_directing**

In [20]:
df_credits_crew['crew'] = df_credits_crew['crew'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df_credits_crew['crew_name_directing'] = df_credits_crew['crew'].apply(
    lambda crew: next((member['name'] for member in crew if member['job'] == 'Director'), None) if isinstance(crew, list) else None
)

In [21]:
df_credits_crew = df_credits_crew.drop(columns=['cast','crew'])

### **Extraccion cast_name**

In [22]:
df_credits_cast['cast'] = df_credits_cast['cast'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df_credits_cast['cast_name'] = df_credits_cast['cast'].apply(
    lambda x: [member['name'] for member in x if isinstance(member, dict) and 'name' in member] if isinstance(x, list) else []
)

In [23]:
df_credits_cast = df_credits_cast.drop(columns=['cast','crew'])


In [33]:
df_credits_crew = df_credits_crew.drop_duplicates(subset=['id'])

In [37]:
df_credits_crew.to_csv('df_credits_crew.csv',index=False)

In [35]:
df_credits_cast = df_credits_cast.drop_duplicates(subset=['id'])

In [39]:
df_credits_cast.to_csv('df_credits_cast.csv',index=False)

In [40]:
df_movies_limpio.to_csv('df_movies_limpio.csv',index=False)

In [31]:
df_movies_limpio = df_movies_limpio.drop_duplicates(subset=['id'])

In [32]:
df_movies_limpio.shape

(45346, 16)